In [1]:
import pandas as pd
import numpy as np
import traceback
import time

from datetime import datetime

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.functions import col, udf, array, count
from pyspark.sql.functions import broadcast,coalesce, lit


OBJECTIVES:

In this challenge, we would like to gather statistics on the number of parking violations (tickets) per street
segment in NYC over the past 5 years. In particular, for each street segment in NYC, we would like to have the
following:
1. The total number of parking violations for each year from 2015 to 2019.
2. The rate that the total number of violations change over the years using Ordinary Least Squares.

The street address is provided through the House Number; Street Name; and Violation County field.
For the parking violations data set, the Issue Date field should be used to determine which year a violationbelongs to.

In [2]:
sc = SparkContext()
spark = SparkSession(sc)

In [3]:
streets = "nyc_cscl.csv"
violations = "nyc_parking_violation/*.csv"

In [4]:
# streets = "hdfs:///tmp/bdm/nyc_cscl.csv"
# violations = "hdfs:///tmp/bdm/nyc_parking_violations/"

In [5]:
start_time = time.time()

In [6]:
def to_upper(string):
    if string is None:
        return None
    return string.strip().upper()

def get_county_code(county):
    if county is not None:
        # Boro codes: 1 = MN, 2 = BX, 3 = BK, 4 = QN, 5 = SI
        if county.startswith("M") or county.startswith("N"):
            return 1
        if county in ['BRONX', 'BX', 'PBX']:
            return 2
        if county in ['BK', 'K', 'KING', 'KINGS']:
            return 3
        if county.startswith('Q'):
            return 4
        if county == 'R' or county == 'ST':
            return 5
    return -1

def get_year(string): 
    data_val = datetime.strptime(string.strip(), '%m/%d/%Y')    
    return data_val.year

def get_house_number(house_val):
    if house_val is None:
        return None
    if type(house_val) is int:
        return house_val
    elems = house_val.split("-")
    new_val = "".join(elems)
    if new_val.isdigit():
        return int(new_val)
    else:
        return None
    
def get_street_number(street_val):
    if street_val is None:
        return 0
    if type(street_val) is int:
        return street_val
    elems = street_val.split("-")
    new_val = "".join(elems)
    if new_val.isdigit():
        return int(new_val)
    else:
        return 0


get_street_number_udf = udf(get_street_number)
get_county_code_udf = udf(get_county_code)
get_year_udf = udf(get_year)
to_upper_udf = udf(to_upper)

In [7]:
def get_violations_df(violations_file, spark):
    violations_df = spark.read.csv(violations_file,header=True, inferSchema=True)

    violations_df = violations_df.select("Violation County", "House Number", "Street Name", "Issue Date")

    violations_df = violations_df.filter((violations_df['Violation County'].isNotNull()) 
                                         & (violations_df['House Number'].isNotNull()) 
                                         & (violations_df['Street Name'].isNotNull()) 
                                         & (violations_df['Issue Date'].isNotNull())
                                        )

    violations_df = violations_df.withColumn('Violation County', get_county_code_udf(violations_df['Violation County']))
    violations_df = violations_df.withColumn('House Number', get_street_number_udf(violations_df['House Number']))
    violations_df = violations_df.withColumn('Street Name', to_upper_udf(violations_df['Street Name']))
    violations_df = violations_df.withColumn('Issue Date', get_year_udf(violations_df['Issue Date']))

    violations_df = violations_df.withColumnRenamed("Violation County","COUNTY")
    violations_df = violations_df.withColumnRenamed("House Number","HOUSENUM")
    violations_df = violations_df.withColumnRenamed("Street Name","STREETNAME")
    violations_df = violations_df.withColumnRenamed("Issue Date","YEAR")

    violations_df = violations_df.where(violations_df.YEAR.isin(list(range(2015,2020))))
    violations_df = violations_df.repartition(5,'COUNTY')
    return violations_df

In [8]:
def get_streets_df(streets_file, spark):

    streets_df = spark.read.csv(streets_file, header=True, inferSchema=True)

    streets_df = streets_df.select("PHYSICALID","BOROCODE", "FULL_STREE", "ST_LABEL","L_LOW_HN", "L_HIGH_HN", 
                                   "R_LOW_HN", "R_HIGH_HN")

    streets_df = streets_df.withColumn('FULL_STREE', to_upper_udf(streets_df['FULL_STREE']))
    streets_df = streets_df.withColumn('ST_LABEL',   to_upper_udf(streets_df['ST_LABEL']))
    streets_df = streets_df.withColumn('L_LOW_HN',  get_street_number_udf(streets_df['L_LOW_HN']))
    streets_df = streets_df.withColumn('L_HIGH_HN', get_street_number_udf(streets_df['L_HIGH_HN']))
    streets_df = streets_df.withColumn('R_LOW_HN',  get_street_number_udf(streets_df['R_LOW_HN']))
    streets_df = streets_df.withColumn('R_HIGH_HN', get_street_number_udf(streets_df['R_HIGH_HN']))

    streets_df = streets_df.withColumnRenamed("L_LOW_HN","OddLo")
    streets_df = streets_df.withColumnRenamed("L_HIGH_HN","OddHi")
    streets_df = streets_df.withColumnRenamed("R_LOW_HN","EvenLo")
    streets_df = streets_df.withColumnRenamed("R_HIGH_HN","EvenHi")
    
    streets_df = streets_df.repartition(5, 'BOROCODE')
    return streets_df

In [9]:
violations_df = get_violations_df(violations, spark)
streets_df = get_streets_df(streets, spark)

In [12]:
# violations_simp = pd.DataFrame(violations_df.head(5), columns=violations_df.columns)
# violations_simp

In [13]:
# streets_simp = pd.DataFrame(streets_df.head(5), columns=streets_df.columns)
# streets_simp

In [14]:
streets_df = streets_df.alias('s')
violations_df = violations_df.alias('v')

In [15]:
merged_df = (
    violations_df.join(
        broadcast(streets_df),
        ((col("s.BOROCODE") == col("v.COUNTY")) &
        (
            (col("s.FULL_STREE") == col("v.STREETNAME")) | 
            (col("s.ST_LABEL") == col("v.STREETNAME"))
        ) &
        (
            ((col("v.HOUSENUM") % 2 == 0)  & (col("v.HOUSENUM") >= col("s.EvenLo")) & (col("v.HOUSENUM") <= col("s.EvenHi"))) |  
            ((col("v.HOUSENUM") % 2 == 1)  & (col("v.HOUSENUM") >= col("s.OddLo"))  & (col("v.HOUSENUM") <= col("s.OddHi")))
        )
    ), how='inner')
).select(col("s.PHYSICALID"),col("v.YEAR"))


In [16]:
merged_df = merged_df.alias('m')
merged_df = merged_df.groupBy("m.PHYSICALID", "m.YEAR").agg(count("*").alias("YEAR_COUNT"))
# merged_df.show(10)

In [17]:
merged_df.createOrReplaceTempView("merged_results")

In [18]:
summaries = spark.sql(
    "select m.PHYSICALID, " +
    "MAX(CASE WHEN (YEAR = 2015) THEN YEAR_COUNT ELSE 0 END) AS COUNT_2015, " +
    "MAX(CASE WHEN (YEAR = 2016) THEN YEAR_COUNT ELSE 0 END) AS COUNT_2016, " +
    "MAX(CASE WHEN (YEAR = 2017) THEN YEAR_COUNT ELSE 0 END) AS COUNT_2017, " +
    "MAX(CASE WHEN (YEAR = 2018) THEN YEAR_COUNT ELSE 0 END) AS COUNT_2018, " +
    "MAX(CASE WHEN (YEAR = 2019) THEN YEAR_COUNT ELSE 0 END) AS COUNT_2019  " +
    "from merged_results m  " +
    "group by m.PHYSICALID " +
    "order by m.PHYSICALID "
)
summaries.show(10)

+----------+----------+----------+----------+----------+----------+
|PHYSICALID|COUNT_2015|COUNT_2016|COUNT_2017|COUNT_2018|COUNT_2019|
+----------+----------+----------+----------+----------+----------+
|        29|         5|         0|         0|         0|         0|
|        30|         5|         0|         0|         0|         0|
|        50|         8|         0|         0|         0|         0|
|        58|         5|         0|         0|         0|         0|
|        62|         6|         0|         0|         0|         0|
|        66|         6|         0|         0|         0|         0|
|        67|        26|         0|         0|         0|         0|
|       116|         8|         0|         0|         0|         0|
|       120|         4|         0|         0|         0|         0|
|       125|         8|         0|         0|         0|         0|
+----------+----------+----------+----------+----------+----------+
only showing top 10 rows



In [19]:
def getOLS(values):
    import statsmodels.api as sm
    X = sm.add_constant(np.arange(len(values)))
    fit = sm.OLS(values, X).fit()
    coef = fit.params[0]
    return float(coef)

getOLS_udf = udf(getOLS)

summaries = summaries.withColumn('OLS_COEF', 
                getOLS_udf(array('COUNT_2015', 'COUNT_2016', 'COUNT_2017', 'COUNT_2018', 'COUNT_2019')))
summaries = summaries.alias('f')

In [20]:
summaries.show()

+----------+----------+----------+----------+----------+----------+------------------+
|PHYSICALID|COUNT_2015|COUNT_2016|COUNT_2017|COUNT_2018|COUNT_2019|          OLS_COEF|
+----------+----------+----------+----------+----------+----------+------------------+
|        29|         5|         0|         0|         0|         0|               3.0|
|        30|         5|         0|         0|         0|         0|               3.0|
|        50|         8|         0|         0|         0|         0|               4.8|
|        58|         5|         0|         0|         0|         0|               3.0|
|        62|         6|         0|         0|         0|         0|3.5999999999999996|
|        66|         6|         0|         0|         0|         0|3.5999999999999996|
|        67|        26|         0|         0|         0|         0|              15.6|
|       116|         8|         0|         0|         0|         0|               4.8|
|       120|         4|         0|         

In [21]:
streets_df = streets_df.select(col("s.PHYSICALID")) \
                    .join(summaries, "PHYSICALID", how='left') \
                    .distinct() \
                    .orderBy("PHYSICALID") \

streets_df = streets_df.withColumn("COUNT_2015",coalesce("COUNT_2015", lit(0))) 
streets_df = streets_df.withColumn("COUNT_2016",coalesce("COUNT_2016", lit(0))) 
streets_df = streets_df.withColumn("COUNT_2017",coalesce("COUNT_2017", lit(0))) 
streets_df = streets_df.withColumn("COUNT_2018",coalesce("COUNT_2018", lit(0))) 
streets_df = streets_df.withColumn("COUNT_2019",coalesce("COUNT_2019", lit(0))) 
streets_df = streets_df.withColumn("OLS_COEF",  coalesce("OLS_COEF", lit(0.0))) 


In [22]:
streets_df.show()

+----------+----------+----------+----------+----------+----------+--------+
|PHYSICALID|COUNT_2015|COUNT_2016|COUNT_2017|COUNT_2018|COUNT_2019|OLS_COEF|
+----------+----------+----------+----------+----------+----------+--------+
|         3|         0|         0|         0|         0|         0|     0.0|
|         5|         0|         0|         0|         0|         0|     0.0|
|         6|         0|         0|         0|         0|         0|     0.0|
|         8|         0|         0|         0|         0|         0|     0.0|
|        14|         0|         0|         0|         0|         0|     0.0|
|        23|         0|         0|         0|         0|         0|     0.0|
|        24|         0|         0|         0|         0|         0|     0.0|
|        25|         0|         0|         0|         0|         0|     0.0|
|        29|         5|         0|         0|         0|         0|     3.0|
|        30|         5|         0|         0|         0|         0|     3.0|

In [23]:
print("--- %s seconds ---" % (time.time() - start_time))

--- 554.238463640213 seconds ---


In [25]:
# streets_df.write.csv('TODO', header=False)

import BDM_Final

In [2]:
%run -i BDM_Final.py output

Output Path:  output


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local[*]) created by __init__ at C:\Users\erikl\OneDrive\Documents\NYU_CUSP\Spring2020\BigDataMA\big-data-management\Final\BDM_Final.py:143 